In [1]:
import re
import requests
import time
import pandas as pd
from fake_useragent import UserAgent

url = 'http://www.hao123.com/'
ua = UserAgent()
headers = {'User-Agent':ua.random}

resp = requests.get(url,headers=headers)
data = resp.text
urls = re.findall(r'href=(http.*?)',data)
print(len(urls))

df = pd.DataFrame()
#1111
df['url'] = urls[:1000]
df.to_csv('TestUrls.csv',index=None)

#装饰器，打印执行时间
def  gettime(func):
    def warapper(*args, **kwargs):
        print('='*50)
        print(func.__name__,'Start...')
        starttime = time.time()
        func(*args)
        endtime = time.time()
        spendtime = endtime - starttime
        print(func.__name__,'End...')
        print('Spend',spendtime,'s totally')
        print('='*50)
    return warapper

#测试网址
def get_urls_from_files(n):
    df = pd.read_csv('TestUrls.csv')
    urls = list(df['url'][:n])
    return urls

#请求并解析网页内容
def getdata(url,retries = 3):
    #print('正在下载:',url)
    headers = {}
    try:
        html=requests.get(url,headers=headers)
        # print(html)
    except requests.exceptions.ConnectionError as e:
        # print('下载出错[Connectionerror]:',e)
        html = None
    if (html!=None and 500 <= html.status_code < 600 and retries):
        retries -= 1
        print('服务器错误，正在重试')
        getdata(url,retries)
        data = html.text
    else:
        data = None
    return data

# 串行
@gettime
def Mynormal():
    for url in urls:
        getdata(url)

# 进程池
def MyprocessPool(num = 10):
    pool = Pool(num)
    results = pool.map(getdata,urls)
    pool.close()
    pool.join()
    return results

# 多线程
@gettime
def Mymultithread(max_threads = 10):
    # 对url进行处理
    def urls_process():
        while True:
            try:
                # 从urls末尾抽出一个
                url = urls.pop()
            except IndexError:
                break
            data = getdata(url,retries=3)
            '''
            这里是对网页的提取与储存操作
            '''
    threads = []

# 未达到最大线程并且仍然在爬取的时候，可以创建新的线程爬取
    while int(len(threads) < max_threads) and len(urls):
        thread = threading.Thread(target = urls_process)
        # print('创建线程',thread.getName())
        thread.start()
        threads.append(thread)
    for thread in threads:
        thread.join()

# 线程池
@gettime
def Myfutures(num_of_max_works=10):
    with concurrent.futures.ThreadPoolExecutor(max_workers = num_of_max_works) as executor:
        executor.map(getdata,urls)

if __name__ == '__main__':
    # 取100个网页进行测试
    urls = get_urls_from_files(100)
    Mynormal()
    MyprocessPool(10)
    Mymultithread(10)
    Myfutures()


D:\Programing\anaconda\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


7
Mynormal Start...


MissingSchema: Invalid URL 'http': No schema supplied. Perhaps you meant http://http?